# Probabilitstic modelling

This new notebook will use the same description for all processes (classical Boltzmann-Gibbs equilibrium, Sample-Space Reducing processes, Bouchaud-Mézard processes) than the previous one [ref]. Rather than simulating an agent in a particular state and its jumps between states, only the distribution of states will be described (as it is in a quantum state), with its evolution.

> fig : 1/ state(t) = i, tirage v.a. X = (new_state, jump_time) --> state(t+jump_time) = new_state
>          loi de v.a. X --> lois exp paramètres 'taux de transition' (dim=temps**-1)
>       2/ state(t) = [a1, a2, ..., ad], state(t+dt) = state(t) + Mat.dot(state(t))*dt
>          master equation

This enables us to use the same description for all upcited models, as well as using non-renormalized states (where the matrix is not stochastic any more, and the total density is more than 1). This is usefull for modelling the BM model as well as variants of the canonical SSR process, and corresponds to probability 'sources' and 'sinks'.

> Ex : ajouter du bruit --> norme(state) = exp((m+sgm2)t)

Our framework is still the modelling of wealth exchange and inequality among a population.

## General description

### Formally

All models will rely on the same general description.
We will study the distribution of a certain quantity $x$ over a finite number of (eingen)states $0, 1, 2, ..., D-1$, and its dynamics. The density vector obeys to a simple first-order linear differential equation, corresponding to the evolution of a Markovian system :
$$\forall \, i \in \lbrace 0, 1, ..., D-1 \rbrace, \qquad \dfrac{\mathrm{d}x_{i}}{\mathrm{d}t} = \sum_{j=0}^{D-1} J_{ij}x_{j}$$
This defines the *master equation* for $\mathbf{X} = (x_i)_{0 \, \leqslant \, i \, < D}$ and $\mathbf{J} = (J_{ij})_{0 \, \leqslant \, i,j \, < \, D}$ reading :
$$\dfrac{\mathrm{d}\mathbf{X}(t)}{\mathrm{d}t} = \mathbf{J}(t)\mathbf{X}(t)$$
Here the state is not normalized and no longer represents the density of state of a single-particle), and the matrix $\mathbf{J}$ is not always stochastic. Also, it is important to note that *both* $\mathbf{X}$ *and* $\mathbf{J}$ are dependant of time : $\mathbf{J}$ can have a noisy component for instance :
$$\mathbf{J}(t) = \mathbf{J_0} + \mathbf{\Xi}(t)$$

### Implementation

We will have the following correspondance for variable names :
- `float x` will be a component $x_i(t)$ of a state in the eigenbasis, 
- `ndarray state` will be the current (macro)state of the particle $\mathbf{X}(t)$,
- `ndarray states` will be the whole time-dependant state $\mathbf{X}$ (a function of time),
- `int dim` will be the number of dimensions $D$, `int i` and `int j` will be indexes refering to the dimension ($0 \, \leqslant \, i,j \, < D$), 
- `int t` will be an index refering to the time coordinate and be comprised in the timeframe $[0 \,, \, T[$ (time horizon `int T`),
- `ndarray J_0` will be the static component $\mathbf{J_0}$ of the driving matrix and `ndarray J` will be the full time-dependant matrix $\mathbf{J}(t)$. Its values at all time $t$ will not be stored,
- `ndarray xi` will be the random and time-dependant component $\mathbf{\Xi}(t)$ of the driving matrix, and `ndarray xis` will be the whole time-varying function $\mathbf{\Xi}$ associated to it,
- ...

## The models

### The canonical ensemble

Lets consider a system of a large number of particles in a box where $D$ energy levels are accessible, at temperature $T$ (`float temp`).

A macrostate of the system can be characterized by the (relative) number of particles in each state $i$ at time $t$ :
$$\forall \, i \in \lbrace 0, ..., D-1 \rbrace, \qquad x_i(t) = \dfrac{\text{# of particles in state }i}{\text{# of particles}}$$
The system will be characterized by its master equation :
$$\forall \, i \in \lbrace 0, ..., D-1 \rbrace, \qquad \dfrac{\mathrm{d}x_{i}(t)}{\mathrm{d}t} = \sum_{j\neq i} J_{ij}(t)x_{j}(t) - \sum_{j\neq i} J_{ji}(t)x_{i}(t)$$

Out-of-equilibrium, the system will behave in a complicated manner where each transition rate $J_{ij}(t)$ is not clearly defined, and is mainly driven by the precise interactions of each particles (is a function of $(\text{# of particles}) \times D$ parameters).

However, at equilibrium the precise distribution of particles in each state $i$ is know, it the *Boltzmann distribution* :
$$\forall \, i \in \lbrace 0, ..., D-1 \rbrace, \qquad x_i(t) = \frac{1}{Z}\exp\left(-\dfrac{E(i)}{k_B T}\right)$$
This distribution is *stationnary*. The transition rates are deduced from the detailed balance :
$$\forall \, i,j \in \lbrace 0, ..., D-1 \rbrace, i\neq j, \qquad J_{ij}x_j = J_{ji}x_i$$
With the stochastic condition giving :
$$\forall \, i \in \lbrace 0, ..., D-1 \rbrace, \qquad J_{ii} = - \sum_{j\neq i} J_{ji}$$

Note : it is important to know that the transition rates $J_ij$ are arbitrary : thermal equilibrium can be reached in any state for any kind of *diffusive part* of the dynamic (upper part of $\mathbf{J}).

### The canonical sample-space reducing (SSR) process

to do

### The Bouchaud-Mézard model for wealth exchange

to do

## To do :
- add the physical dimensions everyhere (top to bottom),
- sort-out the confusion between dimension/state/macrostate/microstate, as well as the confusion between particle/system/etc. In Boltzmann-Gibbs each dimension $i$ is a microstate for each particle, and a state $\mathbf{X}$ is a macrostate of the whole system. In SSR (or description in *General description*) a dimension $i$ is a macrostate and a state $\mathbf{X}$ is (yet) the probability distribution of the macrostates for a single particle (change SSR to many particles ??),
- ...